In [1]:
import os 
from glob import glob 
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt

import tensorflow as tf

%matplotlib inline

In [2]:
tmp_paths = glob('VOCtrainval_06-Nov-2007/VOCdevkit/VOC2007/SegmentationClass/*.png')
len(tmp_paths)

422

In [3]:
image_paths = []
label_paths = []

for lbl_path in tmp_paths:
    img_path = lbl_path.replace('SegmentationClass', 'JPEGImages').replace('.png', '.jpg')
    if os.path.exists(img_path):
        image_paths.append(img_path)
        label_paths.append(lbl_path)

len(image_paths), len(label_paths)

(422, 422)

In [4]:
# 클래스가 뭐 있는지 확인 
class_nums = []

for lbl_path in label_paths:
    label = np.array(Image.open(lbl_path))
    for l in np.unique(label):
        if l not in class_nums:
            class_nums.append(l)
            
class_nums.sort()

In [20]:
class_nums

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 255]

In [19]:
len(class_nums)

22

In [6]:
for lbl_path in label_paths:
    dirname, fname = os.path.split(lbl_path)
    new_dir = os.path.join(dirname, 'new_folder')
    if not os.path.exists(new_dir):
        os.path.mkdir(new_dir)
    label = np.array(Image.open(lbl_path))
    new_label = Image.fromarray(label)
    new_path = os.path.join(new_dir, fname)
    new_label.save(new_path)

In [7]:
label_paths = glob('VOCtrainval_06-Nov-2007/VOCdevkit/VOC2007/SegmentationClass/new_folder/*.png')
label_paths[0], len(label_paths)

('VOCtrainval_06-Nov-2007/VOCdevkit/VOC2007/SegmentationClass/new_folder\\000032.png',
 422)

# Data Preprocess

In [8]:
num_classes = 21
batch_size = 4
input_shape = [256, 256, 3]
drop_rate = 0.3

In [9]:
def data_reader(img_path, lbl_path):
    image_t = tf.io.read_file(img_path)
    image = tf.io.decode_jpeg(image_t)
    
    image = tf.image.resize(image, input_shape[:2])
    image = image / 255
    image.set_shape(input_shape)

    label_t = tf.io.read_file(lbl_path)
    label = tf.io.decode_png(label_t, channels=1)
    label = tf.image.resize(label, input_shape[:2], 'nearest')
    label = tf.where(label==255, tf.zeros_like(label), label)
    label = tf.squeeze(label)
    label.set_shape(input_shape[:2])
    label_onehot = tf.one_hot(label, num_classes)
    
    return image, label_onehot

In [10]:
image, label = data_reader(img_path, lbl_path)
image.shape, label.shape

(TensorShape([256, 256, 3]), TensorShape([256, 256, 21]))

In [11]:
dataset = tf.data.Dataset.from_tensor_slices((image_paths, label_paths))
dataset = dataset.map(data_reader)
dataset = dataset.batch(batch_size)
dataset = dataset.shuffle(100)
dataset = dataset.repeat()

## UNet

In [12]:
from tensorflow.keras import layers

In [21]:
# Build UNet : Encoding
inputs = layers.Input(input_shape)
conv1 = layers.Conv2D(64, 3, padding = 'same', kernel_initializer = 'he_normal')(inputs)
conv1 = layers.BatchNormalization()(conv1)
conv1 = layers.Conv2D(64, 3, padding = 'same', kernel_initializer = 'he_normal')(conv1)
conv1 = layers.BatchNormalization()(conv1)
conv1 = layers.Activation("relu")(conv1)
pool1 = layers.MaxPooling2D(pool_size=(2, 2))(conv1)

conv2 = layers.Conv2D(128, 3, padding = 'same', kernel_initializer = 'he_normal')(pool1)
conv2 = layers.BatchNormalization()(conv2)
conv2 = layers.Activation("relu")(conv2)
conv2 = layers.Conv2D(128, 3, padding = 'same', kernel_initializer = 'he_normal')(conv2)
conv2 = layers.BatchNormalization()(conv2)
conv2 = layers.Activation("relu")(conv2)
pool2 = layers.MaxPooling2D(pool_size=(2, 2))(conv2)

conv3 = layers.Conv2D(256, 3, padding = 'same', kernel_initializer = 'he_normal')(pool2)
conv3 = layers.BatchNormalization()(conv3)
conv3 = layers.Activation("relu")(conv3)
conv3 = layers.Conv2D(256, 3, padding = 'same', kernel_initializer = 'he_normal')(conv3)
conv3 = layers.BatchNormalization()(conv3)
conv3 = layers.Activation("relu")(conv3)
pool3 = layers.MaxPooling2D(pool_size=(2, 2))(conv3)

conv4 = layers.Conv2D(512, 3, padding = 'same', kernel_initializer = 'he_normal')(pool3)
conv4 = layers.BatchNormalization()(conv4)
conv4 = layers.Activation("relu")(conv4)
conv4 = layers.Conv2D(512, 3, padding = 'same', kernel_initializer = 'he_normal')(conv4)
conv4 = layers.BatchNormalization()(conv4)
conv4 = layers.Activation("relu")(conv4)
drop4 = layers.Dropout(drop_rate)(conv4)
pool4 = layers.MaxPooling2D(pool_size=(2, 2))(drop4)

# Bridge Layer
conv5 = layers.Conv2D(1024, 3, padding = 'same', kernel_initializer = 'he_normal')(pool4)
conv5 = layers.BatchNormalization()(conv5)
conv5 = layers.Activation("relu")(conv5)
conv5 = layers.Conv2D(1024, 3, padding = 'same', kernel_initializer = 'he_normal')(conv5)
conv5 = layers.BatchNormalization()(conv5)
conv5 = layers.Activation("relu")(conv5)
drop5 = layers.Dropout(drop_rate)(conv5)

# Decoding
up6 = layers.Conv2DTranspose(1024, 2, padding='SAME', strides=2)(drop5)
merge6 = layers.concatenate([drop4, up6], axis=-1)
conv6 = layers.Conv2D(512, 3, padding = 'same', kernel_initializer = 'he_normal')(merge6)
conv6 = layers.BatchNormalization()(conv6)
conv6 = layers.Activation("relu")(conv6)
conv6 = layers.Conv2D(512, 3, padding = 'same', kernel_initializer = 'he_normal')(conv6)
conv6 = layers.BatchNormalization()(conv6)
conv6 = layers.Activation("relu")(conv6)

up7 = layers.Conv2DTranspose(512, 2, padding='same', strides=2)(conv6)
merge7 = layers.concatenate([conv3, up7], axis=3)
conv7 = layers.Conv2D(256, 3, padding = 'same', kernel_initializer = 'he_normal')(merge7)
conv7 = layers.BatchNormalization()(conv7)
conv7 = layers.Activation("relu")(conv7)
conv7 = layers.Conv2D(256, 3, padding = 'same', kernel_initializer = 'he_normal')(conv7)
conv7 = layers.BatchNormalization()(conv7)
conv7 = layers.Activation("relu")(conv7)

up8 = layers.Conv2DTranspose(256, 2, padding='same', strides=2)(conv7)
merge8 = layers.concatenate([conv2, up8], axis=3)
conv8 = layers.Conv2D(128, 3, padding = 'same', kernel_initializer = 'he_normal')(merge8)
conv8 = layers.BatchNormalization()(conv8)
conv8 = layers.Activation("relu")(conv8)
conv8 = layers.Conv2D(128, 3, padding = 'same', kernel_initializer = 'he_normal')(conv8)
conv8 = layers.BatchNormalization()(conv8)
conv8 = layers.Activation("relu")(conv8)

up9 = layers.Conv2DTranspose(128, 2, padding='same', strides=2)(conv8)
merge9 = layers.concatenate([conv1, up9], axis=3)
conv9 = layers.Conv2D(64, 3, padding = 'same', kernel_initializer = 'he_normal')(merge9)
conv9 = layers.BatchNormalization()(conv9)
conv9 = layers.Activation("relu")(conv9)
conv9 = layers.Conv2D(64, 3, padding = 'same', kernel_initializer = 'he_normal')(conv9)
conv9 = layers.BatchNormalization()(conv9)
conv9 = layers.Activation("relu")(conv9)
conv9 = layers.Conv2D(2, 3, activation='relu', padding='same', kernel_initializer='he_normal')(conv9)
conv10 = layers.Conv2D(num_classes, 1, activation = 'sigmoid')(conv9)

model = tf.keras.Model(inputs=inputs, outputs=conv10)

## Setting Optimizer

In [22]:
def dice(y_true, y_pred):
    epsilon=1e-6
    axes = [1, 2]
    numerator = 2. * tf.reduce_sum(y_pred * y_true, axes)
    denominator = tf.reduce_sum(tf.square(y_pred) + tf.square(y_true), axes)
    
    return tf.reduce_mean(numerator / (denominator + epsilon))

def soft_dice_loss(y_true, y_pred):
    return 1 - dice(y_true, y_pred)

In [23]:
model.compile(loss=soft_dice_loss,
              optimizer=tf.keras.optimizers.Adam(),
              metrics=['accuracy', dice])

In [24]:
image, label = next(iter(dataset))

In [25]:
step_per_epochs = len(image_paths) // batch_size

model.fit(dataset, steps_per_epoch=step_per_epochs)

  1/105 [..............................] - ETA: 0s - loss: 0.9426 - accuracy: 0.0838 - dice: 0.0574

KeyboardInterrupt: 